### Importing modules

In [183]:
import requests
import json
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

### Scraping

In [201]:
#wyciągniecie zupki do web scrapingu
web = requests.get('https://www.beeradvocate.com/beer/styles/').text
soup = BeautifulSoup(web, 'lxml')

#znalezienie klasy z głównymi stylami piwa
beers = soup.find('div', class_ = 'stylebreak')
beer_s_name = beers.find('b').text #nazwa klasy

#wyszukanie klasy z podstylem piwa
beer_groups = beers.find('li')
beer_subs_name = beer_groups.text #nazwa podklasy
beer_subs_link = 'https://www.beeradvocate.com' + beer_groups.a['href'] #link do podklasy do dalszego scrapingu

#dostanie się do konkretnego piwa i browaru z podstylu
subs_web = requests.get(beer_subs_link).text
subs_soup = BeautifulSoup(subs_web, 'lxml')
subs_record = subs_soup.find_all('tr')[3]
subs_lcol = subs_record.find('td', class_ = 'hr_bottom_light') #znalezienie danych tylko z pierwszej kolumny tabeli
subs_lcol_items = subs_lcol.find_all('a', href=True) #wyciągnięcie pozycji z pierwszej kolumny tabeli

subs_names = [item.text for item in subs_lcol_items]
subs_links = ['https://www.beeradvocate.com' + item['href'] for item in subs_lcol_items]

subs_beer_name, subs_brew_name = [subs_names[i] for i, x in enumerate(subs_names)]
subs_beer_link, subs_brew_link = [subs_links[i] for i, x in enumerate(subs_links)]

#scrapowanie konkretnego piwa
beer_web = requests.get(subs_beer_link).text
beer_soup = BeautifulSoup(beer_web, 'lxml')
beer = beer_soup.find('dl', class_ = 'beerstats')
beer_params = beer.find_all('dd', class_ = 'beerstats')
beer_params_text = [item.text.strip() for item in beer_params]
beer_params_text.pop(8)
beer_brewery_name, beer_country, beer_style_rank_glued, beer_abv, beer_score_rank_glued, beer_avg_dev_glued, beer_ratings, beer_active, beer_added, beer_wants, beer_gots = [beer_params_text[i] for i, x in enumerate(beer_params_text)]

#wszystkie wielkości wyscrapowane z piwek
# beer_brewery_name = #Name of brewery which the beer comes from
# beer_country = #Name of country which the beer comes from
beer_style = beer_style_rank_glued.split('Ranked #')[0] #Style of beer
beer_rank_in_style = beer_style_rank_glued.split('Ranked #')[1] #Rank of beer in his style
beer_abv = beer_abv[:-1] #Voltage
beer_score = beer_score_rank_glued.split('Ranked #')[0] #Score on the website
beer_rank = beer_score_rank_glued.split('Ranked #')[1] #Position in the ranking
beer_avg = beer_avg_dev_glued.split('| pDev:')[0].strip() #Average score across all ratings
beer_dev_prc = beer_avg_dev_glued.split('| pDev:')[1].strip()[:-1] #Percentage value of deviation from average score
# beer_ratings = #Number of ratings for this beer
# beer_active = #Current activity status
# beer_added = #When the beer was added
# beer_wants = #Number of users who want this beer
# beer_gots = #Number of users who have this beer

#scrapowanie konkretnego browaru
brewery_web = requests.get(subs_brew_link).text
brewery_soup = BeautifulSoup(brewery_web, 'lxml')
brewery = brewery_soup.find('div', class_ = 'break')

#scrapowanie po tagu 'a'
brewery_a_tag = brewery.find_all('a', href=True)
brewery_a_tag_text = [item.text for item in brewery_a_tag]
brewery_a_tag_links = [item['href'] for item in brewery_a_tag]
if len(brewery_a_tag_text) == 4:
    brewery_a_tag_text.insert(1, "")
    brewery_a_tag_links.insert(1, "")
        
brewery_a_tag_scraped = brewery_a_tag_text[:3] + brewery_a_tag_links[-2:]
brewery_city, brewery_state, brewery_country, brewery_location, brewery_website = [brewery_a_tag_scraped[i] for i, x in enumerate(brewery_a_tag_scraped)]

#rzeczy wyscrapowane z browarów z tagu 'a'
# brewery_city
# brewery_state
# brewery_country
# brewery_location
# brewery_website

#próba wyscrapowania reszty z browarów
brewery_info_list = brewery.text.split('\n') #split on new line symbol
brewery_info_list= [x for x in brewery_info_list if x != ''] #deleting empty strings
brewery_type = brewery_info_list[0].strip().split(',')
brewery_notes = "".join(brewery_info_list).split("Notes:")[-1].strip()
brewery_rest = brewery_info_list[1]
brewery_adress = brewery_rest.split(brewery_city)[0]
brewery_city_state_concat = brewery_city+ ", " + brewery_state
brewery_postal_code = brewery_rest.split(brewery_city_state_concat)[1].split(brewery_country)[0].replace(", ","")
brewery_telephone = brewery_rest.split(brewery_country)[1].split("|")[0].strip()

#rzeczy wyscrapowane z browarów z reszty danych
# brewery_type
# brewery_adress
# brewery_postal_code
# brewery_telephone
# brewery_notes